This notebook archives the code to process the full data set and save the encoded faces in `/data_processed/`

In [3]:
%%capture
!pip install mmcv

In [4]:
import os
import torch
import glob
import time
import numpy as np
import pandas as pd
import mmcv, cv2
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face
from PIL import Image, ImageDraw, ImageFont
from IPython import display
from tqdm import tqdm
from matplotlib import pyplot as plt

In [5]:
Image.__version__

'6.2.1'

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [7]:
# Load face detector
mtcnn = MTCNN(margin=14, keep_all=True, post_process=False, thresholds = [0.9, 0.9, 0.9], device=device).eval()

# Load facial recognition model, but I didn't want to use it yet
resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()

In [8]:
class DetectionPipeline:
    """Pipeline class for detecting faces in the frames of a video file."""
    
    def __init__(self, detector, n_frames=None, batch_size=60, resize=None):
        """Constructor for DetectionPipeline class.
        
        Keyword Arguments:
            n_frames {int} -- Total number of frames to load. These will be evenly spaced
                throughout the video. If not specified (i.e., None), all frames will be loaded.
                (default: {None})
            batch_size {int} -- Batch size to use with MTCNN face detector. (default: {32})
            resize {float} -- Fraction by which to resize frames from original prior to face
                detection. A value less than 1 results in downsampling and a value greater than
                1 result in upsampling. (default: {None})
        """
        self.detector = detector
        self.n_frames = n_frames
        self.batch_size = batch_size
        self.resize = resize
    
    def __call__(self, filename):
        """Load frames from an MP4 video and detect faces.

        Arguments:
            filename {str} -- Path to video.
        """
        # Create video reader and find length
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Pick 'n_frames' evenly spaced frames to sample
        if self.n_frames is None:
            sample = np.arange(0, v_len)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

        # Loop through frames
        faces = []
        frames = []
        for j in range(v_len):
            success = v_cap.grab()
            if j in sample:
                # Load frame
                success, frame = v_cap.retrieve()
                if not success:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)
                
                # Resize frame to desired size
                if self.resize is not None:
                    frame = frame.resize([int(d * self.resize) for d in frame.size])
                frames.append(frame)

                # When batch is full, detect faces and reset frame list
                if len(frames) % self.batch_size == 0 or j == sample[-1]:
                    faces.extend(self.detector(frames))
                    frames = []

        v_cap.release()

        return faces    


def process_faces(faces, resnet):
    # Filter out frames without faces
    faces = [f for f in faces if f is not None]
    faces = torch.cat(faces).to(device)

    # Generate facial feature vectors using a pretrained model
    embeddings = resnet(faces)

    # Calculate centroid for video and distance of each face's feature vector from centroid
#     centroid = embeddings.mean(dim=0)
#     x = (embeddings - centroid).norm(dim=1).cpu().numpy()
    
    return embeddings

1. Sample Videos for speed test
    
    I see 400 videos took minutes 18 minutes on a pair of P100s. There are 292/400 videos with 1 face only, 16 with 2 faces and none with three in every frame. Now I need to run the same procedure on a pair of V100 and see how long it takes. 

In [17]:
# Define face detection pipeline
detection_pipeline = DetectionPipeline(detector=mtcnn, batch_size=60, resize=None, n_frames=30)

# Get all test videos
filenames = glob.glob('data/train_sample_videos/*.mp4')
metadata = pd.read_json('data/train_sample_videos/metadata.json').T

X1 = []
X1_encoded = []
Y1 = []
X2 = []
X2_encoded = []
Y2 = []
X3 = []
X3_encoded = []
Y3 = []
start = time.time()
n_processed = 0
with torch.no_grad():
    for i, filename in tqdm(enumerate(filenames), total=len(filenames)):
        try:
            # Load frames and find faces
            faces = detection_pipeline(filename)
            y = int((metadata.label['data/train_sample_videos/' + metadata.index == filename] == 'REAL') * 1)
            # 1 faces ----------
            if [x.shape[0] for x in faces if x is not None] == [1] * 30:
                # Calculate embeddings
                X1.append(faces)
                X1_encoded.append(process_faces(faces, resnet))
                Y1.append(y)
            # 2 faces ----------   
            if [x.shape[0] for x in faces if x is not None] == [2] * 30:
                # Calculate embeddings
                X2.append(faces)
                X2_encoded.append(process_faces(faces, resnet))
                Y2.append(y)
            # 3 faces ----------   
            if [x.shape[0] for x in faces if x is not None] == [3] * 30:
                # Calculate embeddings
                X3.append(faces)
                X3_encoded.append(process_faces(faces, resnet))
                Y3.append(y)
        except KeyboardInterrupt:
            print('\nStopped.')
            break

        except Exception as e:
            print(e)
        
        n_processed += len(faces)
        print(f'Frames per second (load+detect+embed): {n_processed / (time.time() - start):6.3}\r', end='')


  0%|          | 1/400 [00:03<20:06,  3.02s/it]


  0%|          | 2/400 [00:05<19:43,  2.97s/it]


  1%|          | 3/400 [00:08<19:48,  2.99s/it]


  1%|          | 4/400 [00:11<18:31,  2.81s/it]


  1%|▏         | 5/400 [00:13<18:10,  2.76s/it]

...


 99%|█████████▉| 396/400 [17:38<00:10,  2.71s/it]


 99%|█████████▉| 397/400 [17:40<00:08,  2.68s/it]


100%|█████████▉| 398/400 [17:43<00:05,  2.77s/it]


100%|█████████▉| 399/400 [17:46<00:02,  2.84s/it]


100%|██████████| 400/400 [17:49<00:00,  2.70s/it]

In [ ]:
torch.save(X1_encoded, 'data_processed/sample_train_faces_encoded_1face.pt')
torch.save(Y1, 'data_processed/sample_train_faces_encoded_1face.pt')